In [1]:
import os
import json
import joblib
import pandas as pd
import dask.dataframe as dd

ckpt_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/baseline_models"
data_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/preprocessing/processed/"

patients = pd.read_csv(os.path.join(data_path, "processed_patients.csv"))
diagnoses = pd.read_csv(os.path.join(data_path, "processed_diagnoses.csv"))
prescriptions = pd.read_csv(os.path.join(data_path, "processed_prescriptions.csv"))
procedures = pd.read_csv(os.path.join(data_path, "processed_procedures.csv"))
labevents = pd.read_csv(os.path.join(data_path, "processed_labevents.csv"))
admissions = pd.read_csv(os.path.join(data_path, "processed_admissions.csv"))

In [2]:
patients.drop(columns=["Unnamed: 0"], inplace=True)

In [3]:
diagnoses.disease_category[0]

'Hypertension with complications and secondary hypertension'

In [4]:
import json
import dask.dataframe as dd
from dask.distributed import Client

# Initialize Dask client
client = Client(n_workers=4)  # Adjust based on CPU cores

# Convert to Dask DataFrames
admissions_dd = dd.from_pandas(admissions, npartitions=4)
patients_dd = dd.from_pandas(patients, npartitions=4)
diagnoses_dd = dd.from_pandas(diagnoses, npartitions=8)
procedures_dd = dd.from_pandas(procedures, npartitions=8)
labevents_dd = dd.from_pandas(labevents, npartitions=8)

# Convert to known categoricals (FIX for the error)
diagnoses_dd['disease_category_encoded'] = diagnoses_dd['disease_category_encoded'].astype('category').cat.as_known()
procedures_dd['procedure_category_encoded'] = procedures_dd['procedure_category_encoded'].astype('category').cat.as_known()

# Disease categories features
diag_features = (
    dd.get_dummies(
        diagnoses_dd[['subject_id', 'hadm_id', 'disease_category_encoded']],
        columns=['disease_category_encoded']
    )
    .groupby(['subject_id', 'hadm_id'])
    .sum()
    .reset_index()
)

# Procedure features
proc_features = (
    dd.get_dummies(
        procedures_dd[['subject_id', 'hadm_id', 'procedure_category_encoded']],
        columns=['procedure_category_encoded']
    )
    .groupby(['subject_id', 'hadm_id'])
    .sum()
    .reset_index()
)

# Lab average features
lab_features = (
    labevents_dd.groupby(["subject_id", "hadm_id"])["valuenum_normalized"]
    .mean()
    .reset_index()
    .rename(columns={"valuenum_normalized": "avg_lab_value"})
)

# Merge all features
df = (
    admissions_dd.merge(patients_dd, on="subject_id", how="left")
    .merge(diag_features.compute(), on=["subject_id", "hadm_id"], how="left")  # Compute before merge
    .merge(proc_features.compute(), on=["subject_id", "hadm_id"], how="left")  # Compute before merge
    .merge(lab_features.compute(), on=["subject_id", "hadm_id"], how="left")   # Compute before merge
)

# Handle categorical data
df["gender"] = df["gender"]
df["ethnicity"] = df["ethnicity"]
df["admission_type"] = df["admission_type"].map_partitions(lambda s: s.astype("category").cat.codes, meta=('admission_type', 'int8'))

# Fill NA values and compute final result
df = df.fillna(0).compute()

# Clean up
client.close()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:46165
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39551'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41345'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40491'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43491'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:40951 name: 3
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40951
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:52274
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:37403 name: 0
INFO:

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

# Group all prescribed drugs per admission
target_series = prescriptions.groupby(['subject_id', 'hadm_id'])['drug'].apply(list).reset_index()

# Merge with your Dask-processed DataFrame
df = df.merge(target_series, on=['subject_id', 'hadm_id'], how='left')

# Fill any NaNs in drug column with empty lists
df['drug'] = df['drug'].apply(lambda x: x if isinstance(x, list) else [])

# Fit MultiLabelBinarizer to **all drug names**
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['drug'])

In [6]:
# Save binarizer
joblib.dump(mlb, os.path.join(ckpt_path, "mlb_encoder.pkl"))

['/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/baseline_models/mlb_encoder.pkl']

In [7]:
X = df.drop(columns=[
    'subject_id', 'hadm_id', 'admittime', 'dischtime', 'drug'  # or embedding columns if doing regression
], errors='ignore')

## Multi-layered Perceptron

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# ------------------------ 1. Train/Val/Test Split ------------------------

# First split: Train (70%) vs Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Second split: Validation (15%) vs Test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# ------------------------ 2. Convert to Torch Tensors ------------------------

X_train_tensor = torch.tensor(X_train.values if hasattr(X_train, 'values') else X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.values if hasattr(X_val, 'values') else X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.values if hasattr(X_test, 'values') else X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# ------------------------ 3. Wrap in TensorDataset ------------------------

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

# ------------------------ 4. Create DataLoaders ------------------------

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("DataLoaders ready.")
print("Train:", len(train_loader))
print("Val  :", len(val_loader))
print("Test :", len(test_loader))

DataLoaders ready.
Train: 646
Val  : 139
Test : 139


In [9]:
import torch.nn as nn

class DeepMultiLabelNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DeepMultiLabelNet, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [10]:
input_dim = X_train_tensor.shape[1]
output_dim = y_train_tensor.shape[1]

model = DeepMultiLabelNet(input_dim=input_dim, output_dim=output_dim)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load best model (example: best F1)
model.load_state_dict(torch.load(os.path.join(ckpt_path, "best_f1_model.pth")))
model.to(device)

DeepMultiLabelNet(
  (net): Sequential(
    (0): Linear(in_features=472, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.2, inplace=False)
    (16): Linear(in_features=128, out_features=4525, bias=True)
    (17): Sigmoid()
  )
)

### Train

In [11]:
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 1000

# Track best metrics
best_acc = 0.0
best_f1 = 0.0
best_auc = 0.0

for epoch in range(epochs):
    # ---------- Training ----------
    model.train()
    train_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        preds = model(batch_X)
        loss = criterion(preds, batch_y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # ---------- Validation ----------
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for val_X, val_y in val_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)

            preds = model(val_X)
            loss = criterion(preds, val_y)
            val_loss += loss.item()

            all_preds.append(preds.cpu().numpy())
            all_targets.append(val_y.cpu().numpy())

    # Stack and threshold
    y_pred = np.vstack(all_preds)
    y_true = np.vstack(all_targets)
    y_pred_bin = (y_pred >= 0.5).astype(int)

    # Metrics
    acc = accuracy_score(y_true, y_pred_bin)
    f1 = f1_score(y_true, y_pred_bin, average='micro')
    try:
        auc = roc_auc_score(y_true, y_pred, average='micro')
    except ValueError:
        auc = 0.0  # handle case when only one class present

    # Logging
    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"🔹 Train Loss: {train_loss:.4f} | 🔸 Val Loss: {val_loss:.4f}")
    print(f"📊 Accuracy : {acc:.4f} | F1 Micro: {f1:.4f} | ROC-AUC: {auc:.4f}")

    # ---------- Save Best Checkpoints ----------
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), os.path.join(ckpt_path, "best_acc_model.pth"))
        print(f"✅ Saved best Accuracy model")

    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), os.path.join(ckpt_path, "best_f1_model.pth"))
        print(f"✅ Saved best F1 model")

    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), os.path.join(ckpt_path, "best_auc_model.pth"))
        print(f"✅ Saved best AUC model")

KeyboardInterrupt: 

### Evaluate

In [12]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    all_preds = []
    all_targets = []
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            all_preds.append(outputs.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())

    # Stack and threshold
    y_pred = np.vstack(all_preds)
    y_true = np.vstack(all_targets)
    y_pred_bin = (y_pred >= 0.5).astype(int)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred_bin)
    f1 = f1_score(y_true, y_pred_bin, average='micro')

    try:
        auc = roc_auc_score(y_true, y_pred, average='micro')
    except ValueError:
        auc = 0.0  # Not computable if labels are all 0 or 1 for a class

    return {
        "loss": total_loss,
        "accuracy": acc,
        "f1_micro": f1,
        "roc_auc_micro": auc
    }

In [13]:
# Load best model (example: best F1)
model.load_state_dict(torch.load(os.path.join(ckpt_path, "best_f1_model.pth")))
model.to(device)

# Evaluate
results = evaluate_model(model, test_loader, criterion, device)

# Print results
print("\n🧪 Final Test Set Evaluation:")
print(f"Loss       : {results['loss']:.4f}")
print(f"Accuracy   : {results['accuracy']:.4f}")
print(f"F1 Micro   : {results['f1_micro']:.4f}")
print(f"ROC-AUC    : {results['roc_auc_micro']:.4f}")


🧪 Final Test Set Evaluation:
Loss       : 2.1272
Accuracy   : 0.0807
F1 Micro   : 0.5245
ROC-AUC    : 0.9887


## Inference

In [14]:
import json
import joblib
import numpy as np

mappings_path = "/content/drive/MyDrive/AIT/ML/Project/personalized_medical_recommendation/baseline_models/mappings"

# Load mapping files
with open(os.path.join(mappings_path, "admission_type_mapping.json")) as f:
    admission_type_map = json.load(f)

with open(os.path.join(mappings_path, "ethnicity_mapping.json")) as f:
    ethnicity_map = json.load(f)

with open(os.path.join(mappings_path, "disease_category_mapping.json")) as f:
    disease_map = json.load(f)

with open(os.path.join(mappings_path, "procedure_category_mapping.json")) as f:
    procedure_map = json.load(f)

# Load scaler for lab values
scaler = joblib.load(os.path.join(mappings_path, "scaler_valuenum_labevents.pkl"))

In [17]:
# This should be saved from training (or pulled from training df)
all_columns = joblib.load(os.path.join(mappings_path, "feature_columns.pkl"))  # You must have saved this during training
input_vector = np.zeros(len(all_columns))
column_index = {col: idx for idx, col in enumerate(all_columns)}

In [18]:
# Empty input
input_vector = np.zeros(len(column_index))

# Raw input sample
sample = {
    "age": 0.0,
    "length_of_stay": 19.0,
    "gender": 1,  # Assuming 1 = M, 0 = F
    "ethnicity": "WHITE",
    "admission_type": "EMERGENCY",
    "diseases": ["Liveborn", "Other perinatal conditions"],
    "procedures": ["Prophylactic vaccinations and inoculations", "Respiratory intubation and mechanical ventilation"],
    "lab_values": [7.39, 22.0, 0.93]
}

In [19]:
# Basic fields
input_vector[column_index["age"]] = sample["age"]
input_vector[column_index["length_of_stay"]] = sample["length_of_stay"]
input_vector[column_index["gender"]] = sample["gender"]
input_vector[column_index["ethnicity"]] = ethnicity_map[sample["ethnicity"]]
input_vector[column_index["admission_type"]] = admission_type_map[sample["admission_type"]]

# Lab values
lab_avg = np.mean(sample["lab_values"])
lab_scaled = scaler.transform([[lab_avg]])[0][0]
input_vector[column_index["avg_lab_value"]] = lab_scaled

# Diseases (multi-hot)
for disease in sample["diseases"]:
    if disease in disease_map:
        enc_id = disease_map[disease]
        col_name = f"disease_category_encoded_{enc_id}"
        if col_name in column_index:
            input_vector[column_index[col_name]] += 1

# Procedures (multi-hot)
for proc in sample["procedures"]:
    if proc in procedure_map:
        enc_id = procedure_map[proc]
        col_name = f"procedure_category_encoded_{enc_id}"
        if col_name in column_index:
            input_vector[column_index[col_name]] += 1

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [20]:
import torch

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_input = torch.tensor(input_vector, dtype=torch.float32).unsqueeze(0).to(device)

with torch.no_grad():
    pred = model(X_input)  # Already passed through sigmoid if using BCELoss

probs = pred.cpu().numpy()[0]

In [21]:
top_k = 5
top_indices = probs.argsort()[-top_k:][::-1]

# If you used MultiLabelBinarizer
mlb = joblib.load(os.path.join(ckpt_path, "mlb_encoder.pkl"))  # saved during training
predicted_labels = mlb.classes_[top_indices]

print("Top predicted labels (drugs):", predicted_labels)

Top predicted labels (drugs): ['NEO*PO*Ferrous Sulfate Elixer' 'Potassium Chloride' 'Sodium Chloride'
 'NEO*IV*Gentamicin' 'Syringe (Neonatal) *D5W*']
